In [ ]:
import subprocess
import os
import pathlib
import shutil
import math
import sys
import csv

#Set up parameters for the class room environment
rootdir = 
unix_group  = 
instructors = []
students    = {}
with open('userlist.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if (row['Role']=='Instructor'):
            instructors.append(row['Username'])
        if (row['Role']=='Student' and row['Username'] is not ''):
            students[row['Username']] = [row['Name'], row['Group']]
            
            
# pip pandas numpy matplotlib xarray metpy"

# Functions to be defined here
def setup_class(rootdir, instructors=None): #TESTED
    #Install Python packages
    print("Installing packages")
    !{sys.executable} -m pip install pandas numpy matplotlib xarray metpy jupyterlab-hide-code seaborn

    #First create the directory structure
    print("Creating Directories")
    for subdir in ["groups/", "instructors/", "shared/"]:
        dir = rootdir + subdir
        pathlib.Path(dir).mkdir(parents=True, exist_ok=True)
        link = rootdir+'materials/'+subdir
        print(dir,link)
        try:
            pathlib.Path(link).unlink()
        except FileNotFoundError:
            pass
        pathlib.Path(link).symlink_to( dir )
    
    dir=rootdir+"/instructors/"
    for subdir in ['submissions','submissions/queue', 'submissions/groups', 'starting', 'solutions']:
        pathlib.Path(dir+subdir).mkdir(parents=True, exist_ok=True)
            
    #Students have no access to instructors/ and to the current file
    print("Setting Permissions")
    for subfile in ['/instructors', 'toolkit.ipynb']:
        file = rootdir+subfile
        subprocess.run(["setfacl",       "-m", "g:"+unix_group+":-", file])
        subprocess.run(["setfacl", "-d", "-m", "g:"+unix_group+":-", file])
        subprocess.run(["setfacl",       "-m", "g::-", file])
        subprocess.run(["setfacl", "-d", "-m", "g::-", file])
        subprocess.run(["setfacl",       "-m", "u::-", file])
        subprocess.run(["setfacl", "-d", "-m", "u::-", file])

    #Set up instructor privileges
    for instructor in instructors:
        add_instructor(rootdir, instructor)
        

def add_instructor(rootdir, instructor):#NOT YET TESTED
    print("Adding Instructor " + instructor)
    #Give Instructor full access to all relevant folders
    for file in  ["instructors/", "shared/", "toolkit.ipynb"]:
        subprocess.run(["setfacl", "-R","-d", "-m", "u:"+instructor+":rwX ", rootdir+file])
        subprocess.run(["setfacl", "-R",      "-m", "u:"+instructor+":rwX ", rootdir+file])
    #For all folders inside the groups folder, instructors have read access:
    for dir in os.listdir(rootdir+"groups/"):
        for subdir in [dir, dir+"/submissions/"]:
            subprocess.run(["setfacl", "-R","-d", "-m", "u:"+instructor+":rX ", subdir])
            subprocess.run(["setfacl", "-R",      "-m", "u:"+instructor+":rX ", subdir])
        for subdir in [dir+"/starting/", dir+"/feedback/"]:
            subprocess.run(["setfacl", "-R",      "-m", "u:"+instructor+":rwX ", subdir])
            subprocess.run(["setfacl", "-R","-d", "-m", "u:"+instructor+":rwX ", subdir])

def add_student(rootdir, user, group):#NOT YET TESTED
    print("Adding Student ", user, " to group ", group)
    dir = rootdir + 'groups/' + group + '/'
    for subdir in [dir, dir+"submissions/"]:
        subprocess.run(["setfacl", "-R","-d", "-m", "u:"+user+":rwX ", subdir])
        subprocess.run(["setfacl", "-R",      "-m", "u:"+user+":rwX ", subdir])
    for subdir in [dir+"starting/", dir+"feedback/"]:
        subprocess.run(["setfacl", "-R","-d", "-m", "u:"+user+":rX ", subdir])
        subprocess.run(["setfacl", "-R",      "-m", "u:"+user+":rX ", subdir])

def add_group(rootdir, group): #TESTED
    print("Adding Group ", group)
    dir = rootdir + 'groups/' + group + '/'
    if pathlib.Path(dir).exists():
        print("Group Exists")
        return
    
    #mkdir group in rootdir/groups/; mkdir starting_point, submission, feedback  
    pathlib.Path(dir).mkdir(parents=True, exist_ok=True)
    pathlib.Path(dir+'starting/').mkdir(parents=True, exist_ok=True)
    pathlib.Path(dir+'submissions/').mkdir(parents=True, exist_ok=True)
    pathlib.Path(rootdir+'/instructors/submissions/groups/'+group).mkdir(parents=True, exist_ok=True)
    
    pathlib.Path(dir+'feedback/').mkdir(parents=True, exist_ok=True)

    #Nobody in the class has access to the group folder
    subprocess.run(["setfacl",       "-m", "g:"+unix_group+":- ", dir])
    subprocess.run(["setfacl", "-d", "-m", "g:"+unix_group+":- ", dir])
    subprocess.run(["setfacl",       "-m", "g::- ", dir])
    subprocess.run(["setfacl", "-d", "-m", "g::- ", dir])

    #The instructors have read access to submission, and write access to starting/ and feedback/
    for user in instructors:
        for subdir in [dir, dir+"submissions/"]:
            subprocess.run(["setfacl", "-R","-d", "-m", "u:"+user+":rX ", subdir])
            subprocess.run(["setfacl", "-R",      "-m", "u:"+user+":rX ", subdir])
        for subdir in [dir+"starting/", dir+"feedback/"]:
            subprocess.run(["setfacl", "-R","-d", "-m", "u:"+user+":rwX ", subdir])
            subprocess.run(["setfacl", "-R",      "-m", "u:"+user+":rwX ", subdir])

def add_groups(rootdir, groups, ids=None): #SOME BRANCHES TESTED
    if (groups=='Users'):
        for key, value in ids.items():
            add_group(rootdir, value[0]) 
            add_student(rootdir, key, value[0])
    elif (groups=='List'):
        for key, value in ids.items():
            add_group(rootdir, value[1])
            add_student(rootdir, key, value[1])
    else:
          print("ERROR: Groups needs to be 'Users', an integer or a dictionary with the group names and members")

def distribute(rootdir, exercises=None, groups=None):#TESTED
    if groups == 'All':
        groups = os.listdir(rootdir+'groups/')
    if exercises == 'All':
        dir = rootdir+'instructors/starting/'
        exercises = os.listdir(dir)
        exercises = [x for x in exercises if pathlib.Path(dir+x).is_file()] #Make sure it is files only

    for exercise in exercises:
        print("Distributing " + exercise + " to groups: ", end="")
        for group in groups:           
            print(group, end="    ") 
            shutil.copy(rootdir+'instructors/starting/'+exercise, rootdir+'groups/'+group+'/starting/'+exercise)
        print("\n")


def harvest(rootdir, exercises=None, groups=None): 
    if groups == 'All':
        groups = os.listdir(rootdir+'groups/')
    for group in groups:
        for f_in in os.listdir(rootdir+'groups/'+group+'/submissions/'):
            src = rootdir+'groups/'+group+'/submissions/'+f_in
            dest = rootdir+'instructors/submissions/groups/'+group+'/'+f_in
            if (((exercises == 'All') or (f_in in exercises)) and not pathlib.Path(src).is_dir()):
                if (not os.path.exists(dest)): #If it doesnt exist, just copy
                    print("Retrieving NEW submission from " + group + ":"+ f_in)
                    shutil.copy2(src, dest)
                    base = os.path.basename(dest)
                    link = rootdir+'instructors/submissions/queue/'+group+'_'+base
                    pathlib.Path(link).symlink_to( dest )

                elif os.stat(src).st_mtime - os.stat(dest).st_mtime > 1: #If it does exist and it is updated since the last harvest, copy with a "resub" tag to it
                    print("Retrieving MODIFIED submission from " + group + ":"+ f_in)
                    lst = dest.split(".")
                    dest_new = ".".join(lst[:-1])+".resub."+lst[-1]
                    if not os.path.exists(dest_new):
                        shutil.copy2(src, dest_new)
                        base = os.path.basename(dest_new)
                        pathlib.Path( rootdir+'instructors/submissions/queue/'+group+'_'+base ).symlink_to( dest_new )
                        os.utime(dest) #Update timestamp on the previous file
                    else:
                        print("Skipping "+ dest_new+" File Exists.")

def check_in_list(file, refs): 
    bool = False
    if (refs == 'All'):
        bool = True
    else:
        for ref in refs:
            if (ref in file):
                bool = True
    return bool    

def give_feedback(rootdir, exercises=None, groups=None): 
    
    srcdir = rootdir+'instructors/submissions/queue/'
    
    for file in os.listdir(srcdir):
        do_cp_gr = check_in_list(file, groups)
        do_cp_ex = check_in_list(file, exercises)
                    
        if (do_cp_gr and do_cp_ex):
            lst = file.split("_") #Everything before the first underscore in the file name is the group name; remove that from the filename, copy it to the feedback folder, and remove the link
            group = lst[0]
            fname_out = "_".join(lst[1:])
            print("Handing back "+ fname_out+ " to " + group)
            dest = rootdir+'groups/'+group+'/feedback/'+ fname_out
            shutil.copy2(srcdir+file, dest)
            pathlib.Path(srcdir+file).unlink()

In [ ]:
setup_class(rootdir=rootdir, instructors=instructors)


In [ ]:
add_groups(rootdir, groups='Users')
distribute(rootdir, exercises='All', groups='All')

In [ ]:
harvest(rootdir=rootdir, exercises='All', groups='All')
# give_feedback(rootdir=rootdir, exercises=['wetbulb.ipynb'], groups='All')